In [ ]:
import numpy as np
import pandas as pd

def del_col_csv(df, output_file, col_del):

    # Delete unused columns 
    df = df.drop(columns=col_del)

    # Create new column for arrival date (date format) and delete row if date is not valid
    df['booking_date'] = pd.to_datetime(df[['arrival_year', 'arrival_month', 'arrival_date']].astype(str).agg('-'.join, axis=1), errors='coerce')
    df.drop(df[df['booking_date'].isnull()].index, inplace=True)
    df.drop(['arrival_year', 'arrival_month', 'arrival_date'], axis=1, inplace=True)

    # Replace 0 with 'N' and 1 with 'Y' in car_parking_space column to match data type
    df['required_car_parking_space'] = df['required_car_parking_space'].replace({0: 'N', 1: 'Y'})

    # Sum the columns no_of_weekend_nights and no_of_week_nights into a new column no_nights
    df['no_of_nights'] = df['no_of_weekend_nights'] + df['no_of_week_nights']
    df.drop(['no_of_weekend_nights', 'no_of_week_nights'], axis=1, inplace=True)

    # Remove 75% of rows where booking_status is 'Canceled'
    canceled_indices = df[df['booking_status'] == 'Canceled'].index
    indices_to_remove = np.random.choice(canceled_indices, size=int(0.75 * len(canceled_indices)), replace=False)
    df.drop(indices_to_remove, inplace=True)

    # Rename columns to match the database attributes name
    df.rename(columns={'Booking_ID': 'booking_id'}, inplace=True)
    df.rename(columns={'type_of_meal_plan': 'meal_plan'}, inplace=True)
    df.rename(columns={'required_car_parking_space': 'car_parking_space'}, inplace=True)

    # Keeping only the first 1000 rows
    df = df.iloc[:1000]

    # Update output file
    df.to_csv(output_file, index=False)
    print(f"Output file has been modified : {output_file}")

df = pd.read_csv("Hotel Reservations.csv", sep=",")  # File to modify
output_file = 'C:\\Users\\jerem\\OneDrive\\Documents\\_ESILV A4\\Advanced Database Management\\Projet_Database\\Hotel Reservations Update.csv'  # Output file with modifications

# Columns to delete (yes, we delete booking_id to make it more consistent with the database)
col_del = ['Booking_ID','room_type_reserved','market_segment_type','repeated_guest','no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled','avg_price_per_room','no_of_special_requests']
del_col_csv(df, output_file, col_del)

import csv

csv_file = 'Hotel Reservations Update.csv'
sql_file = 'insert_booking_tuples.sql'

# Open the CSV file for reading
with open(csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Column names

    # Open the SQL file for writing
    with open(sql_file, 'w', encoding='utf-8') as sqlfile:
        for row in reader:
            values = []  
            for col_name, value in zip(header, row):
                # Specific format for booking_date
                if col_name == 'booking_date':
                    value = f"to_date('{value}', 'YYYY-MM-DD')"  
                else:
                    value = f"'{value}'"
                values.append(value)
                
            sqlfile.write(f"INSERT INTO Booking ({', '.join(header)}) VALUES ({', '.join(values)});\n")
        
print(f"File {sql_file} has been successfully generated.")

Output file has been modified : C:\Users\jerem\OneDrive\Documents\_ESILV A4\Advanced Database Management\Projet_Database\Hotel Reservations Update.csv
File insert_booking_tuples.sql has been successfully generated.
